In [5]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# 1. Загрузка данных

In [6]:
DIR_NAME = '../../data/lab_exp/'
filenames = os.listdir(DIR_NAME)

COLUMNS = ['session', 'Period', 'Subject', 'Group', 'Profit', 'TotalProfit',
       'Type', 'CarType', 'Offer', 'Accept', 'CurProfit']

In [7]:
data = pd.read_excel('../../data/lab_exp/180323_LPM.xls')

# 2. Анализ
## 2.1. Наивные стратегии
Назовем предложение продавца наивным, если (почему?). Сколько было наивных предложений? Как они были распределены по вариантам игры?

In [57]:
def get_naive_offer(df):
    """find naive offer (Offer between 2 and 10)"""
    # only seller
    df = df[df['Type']==1]
    # naive offer
    df = df[(df['Offer'] > 2) & (df['Offer'] < 10)]
    return df

def analyse_naive_offer(filenames):
    """Return 2 DataFrames:
    First - all action with naive offer
    Second - aggregate by filename, session and Subject"""
    df = pd.DataFrame()
    for filename in filenames:
        df_ = get_naive_offer(
            pd.read_excel(DIR_NAME + filename)
        )
        df_.insert(0, 'filename', filename)
        df = df.append(df_)
    df = df[['filename'] + COLUMNS]
    
    gb = df.groupby(
        ['filename', 'session', 'Subject']
    )[['Type']].count().rename(
        columns={'Type':'cnt'}
    )
    
    return (df, gb)

In [58]:
d, g = analyse_naive_offer(filenames)
g

cnt
filename         session Subject     
180323_LPM.xls   1       2          1
180406_LPM1.xlsx 1       6          2
180413_LPM.xlsx  1       3          3
                 2       3         13
180427_LMT.xlsx  3       12         2

## 2.2. Честные продавцы
Назовем продавца честным, если он не пытается продать Лимон по цене, большей 2. Были ли в игре честные или почти честные игроки?

In [13]:
def get_limon_car(df):
    """find naive offer (Offer between 2 and 10)"""
    # only seller
    df = df[df['Type']==1]
    # limon car
    df = df[df['CarType'] == 1]
    return df

def analyse_honest(filenames):
    """Return 2 DataFrames:
    First - all action with naive offer
    Second - aggregate by filename, session and Subject"""
    df = pd.DataFrame()
    for filename in filenames:
        df_ = get_limon_car(
            pd.read_excel(DIR_NAME + filename)
        )
        df_.insert(0, 'filename', filename)
        df = df.append(df_)
    df = df[['filename'] + COLUMNS]
    df['honest'] = (df['Offer'] <= 2).astype(int)
    
    gb = df.groupby(
        ['filename', 'Subject'] # 'session', 
    )[['honest']].agg(['count', 'sum'])
    
    return (df, gb)

In [14]:
(d, g) = analyse_honest(filenames)

In [16]:
g[g[('honest', 'count')] == g[('honest', 'sum')]].head(20)

honest    
                          count sum
filename         Subject           
180323_LPM.xls   1            7   7
                 8           16  16
180406_LPM1.xlsx 1           12  12
                 10           1   1
180420_LMP.xlsx  1           11  11
                 3            6   6

## 2.3. Успешные обманы
Сколько у кого было шансов продать Лимон, сколько раз предложение было честным, а сколько нечестным; сколько раз удалось продать лимон по цене большей, чем 10?

In [ ]:
d, g = get_limon_car(filenames)

In [20]:
# честные - нечестные предложения
d['honest'].value_counts()

0    422
1    200
Name: honest, dtype: int64

In [21]:
d.head(1)

,filename,session,Period,Subject,Group,Profit,TotalProfit,Type,CarType,Offer,Accept,CurProfit,honest
1,180323_LPM.xls,1,1,2,5,12.0,12.0,1,1,8.0,0,0.0,0


In [23]:
# распределение успешных обманов
d[d['Offer'] > 10]['Accept'].value_counts()

0    258
1    157
Name: Accept, dtype: int64

## 2.4. Обидчивые игроки
Были ли такие игроки, которые сначала были честными, а потом обиделись и уже сами стали нечестными?

In [8]:
b = get_limon_car(data)
b

,session,Period,Subject,Group,Profit,TotalProfit,Type,CarType,Offer,Accept,CurProfit
1,1,1,2,5,12.00,12.00,1,1,8.00,0,0.00
8,1,1,9,3,22.00,22.00,1,1,11.00,1,10.00
13,1,2,2,5,12.90,24.90,1,1,1.90,1,0.90
23,1,2,12,4,12.00,24.00,1,1,2.00,0,0.00
31,1,3,8,2,12.50,38.50,1,1,1.50,1,0.50
46,1,4,11,1,22.00,61.00,1,1,11.00,1,10.00
47,1,4,12,4,12.00,48.00,1,1,2.00,0,0.00
49,1,5,2,5,12.90,63.80,1,1,1.90,1,0.90
55,1,5,8,2,12.50,64.00,1,1,1.50,1,0.50
58,1,5,11,1,12.00,73.00,1,1,11.00,0,0.00


In [17]:
a[a.Subject==10].head(20)

,session,Period,Subject,Group,Profit,TotalProfit,Type,CarType,Offer,Accept,CurProfit
9,1,1,10,6,12.5,12.5,2,2,11.5,1,0.5
21,1,2,10,6,12.5,25.0,2,2,11.5,1,0.5
33,1,3,10,6,12.5,37.5,2,2,11.5,1,0.5
45,1,4,10,6,12.5,50.0,2,2,11.5,1,0.5
57,1,5,10,6,22.5,72.5,2,2,1.5,1,10.5
69,1,6,10,6,12.0,84.5,2,2,11.5,0,0.0
81,1,7,10,6,12.0,96.5,2,2,11.5,0,0.0
93,1,8,10,6,12.0,108.5,2,2,11.5,0,0.0
105,1,9,10,6,12.0,120.5,2,2,11.5,0,0.0
117,1,10,10,6,12.0,132.5,2,2,11.5,0,0.0
